In [1]:
import numpy as np
import matplotlib.pyplot as plt
import arviz as az

from ipywidgets import interact
import ipywidgets as ipyw

In [2]:
az.style.use('arviz-whitegrid')

In [3]:
%%HTML
<style>

.CodeMirror {
    width: 100vw;
}

.container {
    width: 99% !important;
}

.rendered_html {
  font-size:0.8em;
}
.rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
     font-size: 100%;
}


body {
  font-family: Ubuntu;
  background: #F0F0F0;
  background-color: #F0F0F0;
}


.reveal h1,
.reveal h2,
.reveal h3,
.reveal h4,
.reveal h5,
.reveal h6 {
  margin: 0 0 20px 0;
  color: #2a2eec;
  font-family: Ubuntu;
  line-height: 0.9em;
  letter-spacing: 0.02em;
  text-transform: none;
  text-shadow: none;
}

.reveal blockquote {
  display: block;
  position: relative;
  background: #fa7c17;
  border-radius: 15px;
  box-shadow: 0px 0px 2px rgba(0, 0, 0, 0.2);
  font-weight: bold;
}

</style>

In [4]:
from traitlets.config.manager import BaseJSONConfigManager
path = "/home/osvaldo/anaconda3/etc/jupyter/nbconfig"
cm = BaseJSONConfigManager(config_dir=path)
cm.update("livereveal", {
              "theme": "serif",
              "transition": "zoom",
              "start_slideshow_at": "selected",
              "controls": "True",
              "progress": "False",
              "shortcut": "False",});


##### <center><img src="img/logo_inst.png" width="700">
<br>
<br>
<br>
<br>
<h1 align="center">Exploratory Analysis of Bayesian models</h1> 
<h2 align="center">How I learned to stop worrying and love Bayesian modeling</h2> 



<br>
<br>
<br>
<br>
<br>

# ArviZ: Exploratory analysis of Bayesian models

<br>
<br>
<center><img src="img/logo_arviz.png" width=400></center>
<br>


* Diagnoses of the quality of the inference
* Model criticism, including evaluations of both model assumptions and model predictions
* <font color = "gray"> Comparison of models, including model selection or model averaging </font>
* <font color = "gray"> Preparation of the results for a particular audience </font>

In [ ]:
# Diagnoses of the quality of the inference

something something approximated methods

## Markov Chain Monte Carlo (MCMC)

<br>

* MCMC methods like Metropolis-Hastings can be used to sample from probability distributions that are difficult to sample directly

* If detailed balance condition is satisfied, there is a theoretical guarantee that MCMC methods will _eventually_ converge to a  stationary distribution. In practice this can be slow for many interesting problems.

* Alternatives to Metropolis-Hastings have been proposed, including methods such as Hamiltonian Monte Carlo and Sequential Monte Carlo.

In [ ]:
IFrame(src='mcmc-demo-master/app.html#RandomWalkMH,standard', width=1200, height=600)

## Trace plots

<br>

    az.plot_trace(trace)


* MCMC samples should not be sensitive to the starting point (ergodicity)
    * So if you sample more than one chain (starting from different places) you should get the same results
* MCMC samples should have the lowest possible autocorrelation  
    * No long regions of monoticity

Trace plots can help diagnose:

* Multimodal posteriors
* Bad intialization
* Difficult sampling topologies (such as Neal's funnel)

## Pathological traces

<br>

<center><img src="img/pathological_traces.png" width=600></center>

## How to fix it

* More samples
* Burn-in
* Change the sampler!
* Reparametrize the model
* Read PyMC3 warning messages ans suggestions!

## Autocorrelation plot

<br>

    az.plot_autocorr(trace)


## Rank plot

<br>
    az.plot_rank()


## $\hat R$ (aka R hat)

* Compares the _between chain_ variance with the _in chain_ variance .

* If all the chains have converged, the variance should be similar across all chains, and the pooled sample of all chains.

* Ideally it should be 1, numbers lower than 1.05 are generally considered OK. 

* It can be interpreted as the overestimation of variance due to MCMC finite sampling. If you continue sampling infinetelly you should get a reduction of the variance of your estimation by a $\hat R$ factor.


In [ ]:
## Effective Sample Size

## Divergences

## Simulation-Based Calibration

In [ ]:
# Model criticism

<center><img src="img/Bayesian_workflow.png" width=900></center>